# **Библиотеки**

In [2]:
import librosa
import zipfile
import io
from scipy.io import wavfile
from google.colab import drive
import numpy as np
import pandas as pd

# **Данные**

# Подгрузка данных

In [3]:
drive.mount('/content/drive')
df=pd.read_csv('drive/MyDrive/Анализ_акустических_событий/train.csv', sep=',',header=None)
dataset = zipfile.ZipFile('drive/MyDrive/Анализ_акустических_событий/audio_train.zip')
labels = df.values.tolist()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Создание словарей с переводом label названий в label цифры и наоборот

In [4]:
label_to_figure = {}
count = 0
for i in range(len(labels)):
    if i == 0:
        pass
    elif labels[i][1] in label_to_figure:
        pass
    else:
        label_to_figure[labels[i][1]] = count
        count += 1

figure_to_label = {}
for i in label_to_figure.items():
    figure_to_label[i[1]] = i[0]

# Достаём wav файлы из ZIP

In [5]:
def read_from_zip(filename):
    with io.BytesIO() as bytes_buff:
        bytes_data = dataset.read(filename)
        bytes_buff.write(bytes_data)
        bytes_buff.seek(0)
        sr, wave = wavfile.read(bytes_buff)
    return sr, wave

# Создаём Mel признаки

In [6]:
import pandas as pd
X = []
Y = []
for i in range(len(labels)//5):
    if not i == 0:
        sr, X_mel = read_from_zip(f'train/{labels[i][0]}')
        vre = np.zeros(480000)
        vre[:X_mel.shape[0]] = X_mel
        X.append(vre)
        Y.append(label_to_figure[labels[i][1]])
X = np.array(X)
Y = np.array(Y)

# **Сохраняем prl**

Не получается, прерывается из-за ОЗУ

In [ ]:
import pickle

output = open('X.pkl', 'wb')
pickle.dump(X, output)
output.close()

output = open('Y.pkl', 'wb')
pickle.dump(Y, output)
output.close()

pkl_file = open('X.pkl', 'rb')
X = pickle.load(pkl_file)
pkl_file.close()

pkl_file = open('Y.pkl', 'rb')
Y = pickle.load(pkl_file)
pkl_file.close()

# **Классификатор**

# Делим данные на траин и валидацию

In [7]:
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from IPython.display import Image
from imutils import paths
import numpy as np
import cv2
import os

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

# Обучение

In [10]:
from sklearn.svm import LinearSVC
model = LinearSVC(random_state = 51, C = 1)
model.fit(X_train, y_train)

LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=51, tol=0.0001,
          verbose=0)

# Тест

In [11]:
predictions = model.predict(X_test)

In [14]:
from sklearn.metrics import f1_score
print(f1_score(y_test, predictions, average='macro'))
print(classification_report(y_test, predictions))

0.06288464469680449
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.00      0.00      0.00        15
           2       0.00      0.00      0.00         4
           3       0.00      0.00      0.00         2
           4       0.60      0.21      0.32        14
           5       0.12      0.08      0.10        12
           6       0.00      0.00      0.00         4
           7       1.00      0.08      0.15        12
           8       0.00      0.00      0.00        19
           9       0.00      0.00      0.00        15
          10       0.00      0.00      0.00         9
          11       0.00      0.00      0.00        11
          12       0.07      0.12      0.09         8
          13       0.00      0.00      0.00        14
          14       0.50      0.36      0.42        11
          15       0.67      0.67      0.67         3
          16       0.14      0.57      0.22         7
       

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
